# 基于文本的推荐系统

#### 学号：3120220948  姓名：刘思雯

### [github地址](https://github.com/black552/DataMining2/tree/main)

##### 1.读取、处理数据

In [10]:
import csv
import random
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
# open the file in the write mode
with open('./data/user_item.csv', 'w', newline='') as f:
    # create the csv writer
    writer = csv.writer(f)
    header = ['用户', '物品','是否相关']
    writer.writerow(header)
    for i in range(200):
        user = '用户' + str(random.randint(1,100))
        item = '物品' + str(random.randint(1, 100))
        if i < 100:
            isrelated = '相关'
        else:
            isrelated = '不相关'
        row = [user, item, isrelated]
        writer.writerow(row)
uicsv = pd.read_csv('./data/user_item.csv',encoding='gb2312')
print(uicsv.head(15)) # 打印前15行

      用户    物品 是否相关
0   用户96  物品24   相关
1   用户11  物品70   相关
2   用户26  物品41   相关
3    用户5   物品1   相关
4   用户53  物品57   相关
5   用户47   物品1   相关
6   用户31  物品99   相关
7   用户93   物品6   相关
8   用户11   物品1   相关
9   用户11  物品10   相关
10  用户64  物品14   相关
11  用户72  物品78   相关
12  用户90  物品69   相关
13  用户56  物品36   相关
14  用户49  物品92   相关


In [11]:
# 去重-去掉用户和物品重复的行
frame=pd.read_csv('./data/user_item.csv',names=['用户','物品','是否相关'],skiprows = [0],encoding='gb2312')
print(frame)
data = frame.drop_duplicates(subset=['用户','物品'], keep='first', inplace=False)
data.to_csv('./data/output.csv', encoding='gb2312')
csvs = pd.read_csv('./data/output.csv',encoding='gb2312')
print(csvs)

       用户    物品 是否相关
0    用户96  物品24   相关
1    用户11  物品70   相关
2    用户26  物品41   相关
3     用户5   物品1   相关
4    用户53  物品57   相关
..    ...   ...  ...
195  用户44  物品62  不相关
196   用户9   物品5  不相关
197  用户81  物品14  不相关
198  用户36  物品82  不相关
199  用户95  物品48  不相关

[200 rows x 3 columns]
     Unnamed: 0    用户    物品 是否相关
0             0  用户96  物品24   相关
1             1  用户11  物品70   相关
2             2  用户26  物品41   相关
3             3   用户5   物品1   相关
4             4  用户53  物品57   相关
..          ...   ...   ...  ...
194         195  用户44  物品62  不相关
195         196   用户9   物品5  不相关
196         197  用户81  物品14  不相关
197         198  用户36  物品82  不相关
198         199  用户95  物品48  不相关

[199 rows x 4 columns]


In [12]:
# 数据归一化
orifile = csv.reader(open('./data/output.csv', 'r',encoding='gb2312'))
tarfile = csv.writer(open('./data/normalization_output.csv', 'w', newline='', encoding='gb2312'))
tarfile.writerow(['user', 'item', 'relative'])
for i in orifile:
    usero = re.sub('[\u4e00-\u9fa5]', '',i[1]) 
    if usero != '':
        usern = float(usero)/ 100
    else:
        usern = 0
    itemo = re.sub('[\u4e00-\u9fa5]', '',i[2]) 
    if itemo != '':
        itemn = int(itemo)/ 100
    else:
        itemn = 0
    relativeo = i[3]
    if  relativeo == '相关':
        relativen = 1
    else:
        relativen = 0
    if usern!=0 and itemn!= 0:
        tarfile.writerow([usern,itemn,relativen])

#### 2.文本挖掘

In [6]:
# 挖掘用户和物品之间的关联信息
data = pd.read_csv('./data/normalization_output.csv', engine='python')
texts = data[['user', 'item']]
target = data['relative']
X_train,X_test,y_train,y_test = train_test_split(texts,target,test_size=0.33,random_state=1)
print(X_train,y_train)

     user  item
69   0.21  0.66
48   0.07  0.89
66   0.12  0.65
54   0.48  0.87
130  0.15  0.88
..    ...   ...
133  0.40  0.22
137  0.28  0.41
72   0.34  0.98
140  0.44  0.02
37   0.71  0.92

[132 rows x 2 columns] 69     1
48     1
66     1
54     1
130    0
      ..
133    0
137    0
72     1
140    0
37     1
Name: relative, Length: 132, dtype: int64


In [4]:
number_of_clusters = 2
model = KMeans(n_clusters=number_of_clusters,
              init='k-means++',
              max_iter=100,
              n_init=1)
yplabel=model.fit_predict(X_train)

In [64]:
labels = model.predict(X_test)
utdict = {}
for i in range(len(labels)):
    if labels[i] == 1:
        user = int(X_test.iloc[i][0] * 100)
        item = int(X_test.iloc[i][1] * 100)
        utdict.setdefault(user, []).append(item)
#         print('用户',int(user), '将推荐物品', int(item))
for item in utdict.items():
    print('用户',item[0], '将推荐物品',' ', end='')
    for i in range(len(item[1])):
        if i == len(item[1])-1:
            print(item[1][i])
        else:
            print(item[1][i], ',', end='')

用户 93 将推荐物品  70 ,62 ,27
用户 98 将推荐物品  48
用户 97 将推荐物品  79 ,96
用户 18 将推荐物品  98
用户 68 将推荐物品  76
用户 87 将推荐物品  78
用户 69 将推荐物品  53
用户 99 将推荐物品  65
用户 54 将推荐物品  59
用户 66 将推荐物品  74
用户 34 将推荐物品  76
用户 74 将推荐物品  83
用户 24 将推荐物品  83
用户 100 将推荐物品  21
用户 64 将推荐物品  79 ,89
用户 60 将推荐物品  91
用户 72 将推荐物品  80
用户 11 将推荐物品  97
用户 95 将推荐物品  98
用户 14 将推荐物品  90
用户 47 将推荐物品  60
用户 78 将推荐物品  88
用户 30 将推荐物品  92
用户 92 将推荐物品  74
